# Synthesize Data 

## 0 - Setup

### 0.1 - Packages 

In [1]:
from numpy import repeat
from ydata.synthesizers.regular import RegularSynthesizer
from ydata.connectors import LocalConnector
from ydata.connectors.filetype import FileType
from ydata.dataset import Dataset

/opt/conda/lib/python3.7/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


## 1 - Load Data

In [2]:
# Read the data (Only if you want to train a new network)
# Initialize the YData's connector
connector = LocalConnector()

# Read the data
data = connector.read_file('data_processed.csv', file_type = FileType.CSV)

## 2 - Synthesization 

### 2.1 - Train the Synthesizer 

In [3]:
# Initialize and Load an alerady trained Synthesizer.
network_synth = RegularSynthesizer(filter_outliers=False)
#network_synth = network_synth.load('trained_model.pkl')
data = data.to_pandas().query("Unusual == 1")
network_synth.fit(Dataset(data))
network_synth.save('trained_model_unb.pkl')

INFO: 2022-03-31 13:24:30,856 [SYNTHESIZER] - Calculating metadata
Number columns considered for synth: 9
INFO: 2022-03-31 13:24:32,385 [SYNTHESIZER] - Synthesizer init.
INFO: 2022-03-31 13:24:32,386 [SYNTHESIZER] - Processing the data prior fitting the synthesizer.


### 2.2 - Sample Data 

In [5]:
# Create the samples and flag them.
synth_sample = network_synth.sample(5000).to_pandas()

# Flag the created samples and assign columns 
synth_sample.index = repeat(999999, len(synth_sample))
synth_sample.columns = data.columns

INFO: 2022-03-31 13:24:56,275 [SYNTHESIZER] - Start generating model samples.


### 3 - Store Data 

In [5]:
connector.write_file(data= synth_sample, path='data_sampled.csv', index=False)